In [2]:
%matplotlib inline

# Standard imports
import copy
import itertools

# Scientific computing imports
import numpy as np
import matplotlib.pyplot as plt
import networkx as nx
import pandas
import seaborn; seaborn.set()

# Import widget methods
from IPython.html.widgets import *

#Student Class

In [ ]:
class Student(object):
    
    def __init__(self, model, school_id, cohort_id, student_id):
        
        #Set link to model, school ID, cohort ID and student ID
        self.model = model
        self.person_id = person_id
        self.school_id = school_id
        self.cohort_id = cohort_id
        self.student_id = student_id
    
    
    def decide_(self):
        
        

#Model Class

* constructor: class constructor, which "initializes" or "creates" the model when we call Model(). This is in the __init__ method.


In [11]:
class Model(object):
    """
    Model Class, which encapsulates the entire behavior in each run of the model
    """
    def __init__(self, num_students, num_colleges, num_high_schools, num_seats, num_cohorts=3, 
               prob_out=.1, min_sticker_price=10000, max_sticker_price=50000, min_net_price=0, 
               max_net_price=50000, min_college_budget=0, max_college_budget=100000, at_prob_l=.2, 
               at_prob_m=.5, at_prob_h=.8, switch_prob=.6):
        
        #Set Model parameters 
        self.num_students = num_students
        self.min_sticker_price = min_sticker_price
        self.max_sticker_price = max_sticker_price
        self.min_net_price = min_net_price
        self.max_net_price = max_net_price
        self.min_college_budget = min_college_budget
        self.max_college_budget = max_college_budget
        self.num_high_schools = num_high_schools
        self.num_colleges = num_colleges
        self.num_seats = num_seats
        self.at_prob_l = at_prob_l
        self.at_prob_m = at_prob_m
        self.at_prob_h = at_prob_h
        self.switch_prob = switch_prob
        
        student_graphs = {} 
        
    def setup_hs_space(self):
        """
        Set up small world networks within highschools
        """
        # school list
        school_list = string.ascii_uppercase[0:num_schools]

        # Iterate
        for school_id in school_list:
            # Set the school ID
            num_cohorts = 3
            prob_out = 0.1

            # Setup empty school dictionary key
            student_graphs[school_id] = {}

            # Iterate through each cohort
            for cohort_id in range(num_cohorts):
                # Create graph randomly
                num_students = numpy.random.randint(50, 500)
                num_friends = 2 + numpy.random.binomial(4, 0.5)
                g = nx.newman_watts_strogatz_graph(num_students, num_friends, prob_out)

                # Create students
                for node_id in g.nodes():
                    # Set a student class as an attribute
                    g.node[node_id] = Student(school_id, cohort_id, node_id)

                # Set school-cohort graph into dictionary
                student_graphs[school_id][cohort_id].node[student_id] = g 

In [12]:
student_graphs

NameError: name 'student_graphs' is not defined